# Imports

In [25]:
import socket
import time
import datetime
import numpy as np

# UDP Testing

In [2]:
IP_ADDRESS = "192.168.0.17"
TCP_Port = 23
UDP_Port = 4660

In [26]:
sock = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)

In [141]:
address = 0x189
readBytes = bytearray([0xFF,0xc0,0xaa,0x01]) + address.to_bytes(4,'big')
print(readBytes)

bytearray(b'\xff\xc0\xaa\x01\x00\x00\x01\x89')


In [126]:
bin(0x80)

'0b10000000'

In [143]:
address = 0x189
data = 0x40
writeBytes = bytearray([0xFF,0x80,0xaa,0x01]) + address.to_bytes(4,'big') + bytes([data])
print(writeBytes)

bytearray(b'\xff\x80\xaa\x01\x00\x00\x01\x89@')


In [145]:
sock.sendto(readBytes,(IP_ADDRESS,UDP_Port))
rec = sock.recv(1024)

print(f"Header: {rec[:8]}\nData: {rec[8:]}")

Header: b'\xff\xc8\xaa\x01\x00\x00\x01\x89'
Data: b'@'


In [48]:
len(rec[0])

16

In [21]:
rec[0]

b'\xff\xc8\x00\xff\x00\x00\x01\xb5\x00\x90\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xea`\x13\x88+\xf2\x01\xf4\x13\x87\t\x7f\x7f\x7f\x7f\x7f\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8'

In [12]:
bin(0x8)

'0b1000'

In [2]:
0x01

1

In [171]:
(bytes([0xff,0x24]) + bytes([0x1b])).hex(':')

'ff:24:1b'

In [173]:
test = 1
isinstance(test,list)

False

In [53]:
send = bytearray([0xff,0x40,0x80]) + bytearray([0x34])
print(send)

bytearray(b'\xff@\x804')


In [61]:
bytes([0x0])

b'\x00'

In [56]:
hex(send[2])

'0x80'

In [62]:
def UDP_Neunet(IP='192.168.0.17',port=4660,ID=None,length=1,address=0x80,data=None,printOutput=False):
        modeRead,modeWrite = 0xc0, 0x80
        if ID is None:
                ID = np.random.randint(0,255)
        sendBytes = bytearray([0xff,0x00,ID,length]) + address.to_bytes(4,'big')
        if data is not None:
                if not isinstance(data,list):
                        data = [data]
                sendBytes += bytearray(data)
                sendBytes[1] = modeWrite
                sendBytes[3] = len(data) #Make the length match sent byte length automatically
        else:
                sendBytes[1] = modeRead
                
        with socket.socket(socket.AF_INET,socket.SOCK_DGRAM) as sock:
                sock.sendto(sendBytes,(IP,port))
                recvData = sock.recv(1024)
        flagByte = recvData[1] % 2**4
        if flagByte % 2**1:
                raise ConnectionRefusedError('Bus error! Check the format of the sent packet.',
                                             f"Send: {sendBytes.hex(':')}\n",
                                             f"Header : {recvData[:8].hex(':')}\n",
                                             f"Data : {recvData[8:].hex(':')}"
                                             )        
        if printOutput:
                print(f"Send: {sendBytes.hex(':')}\n",
                      f"Header : {recvData[:8].hex(':')}\n",
                      f"Data : {recvData[8:].hex(':')}")
        return recvData

In [57]:
instrument_time()

b'\x1d[\xa0&p'

In [68]:
hex(0b10000000)

'0x80'

In [ ]:

def startupUPD():
    #Set time to 32-bit mode
    responseByte = UDP_Neunet(address=0x18a,data=0x80)

    #First send the computer time to the instrument
    instrument_time(UDP_Neunet(address=0x190,length=5),mode='datetime')
    responseByte = UDP_Neunet(address=0x190,data=instrument_time())
    instrument_time(UDP_Neunet(address=0x190,length=5),mode='datetime')

    #Set event memory read mode
    responseByte = UDP_Neunet(address=0x186,data=bytes([0x00,0x00]))

    #Set one-way mode
    responseByte = UDP_Neunet(address=0x1b5,data=0x80)

    # #FIFO buffer cleared
    # responseByte = UDP_Neunet(address=0x189,data=0x60)

In [60]:
hex(0b1100000)

'0x60'

In [3]:
bin(0x40)

'0b1000000'

In [88]:
UDP_Neunet(address=0x1b0,length=6)

Send: ff:c0:53:06:00:00:01:b0
 Header : ff:c8:53:06:00:00:01:b0
 Data : 03:20:00:80:00:00


b'\xff\xc8S\x06\x00\x00\x01\xb0\x03 \x00\x80\x00\x00'

In [221]:
output = b'\xff\xc8S\x06\x00\x00\x01\xb0\x03 \x00\x80\x00\x00'
bin(output[1] % 2**4)

'0b1000'

In [71]:
UDP_Neunet(address=0x1b3)

KeyboardInterrupt: 

In [94]:
readOnlyBytesStartup = UDP_Neunet(length=(0x9b-0x80),address=0x80).hex(':')
with open('readOnlyBytes.dat','w') as openFile:
    openFile.write(readOnlyBytesStartup)

Send: ff:c0:88:1b:00:00:00:80
 Header : ff:c8:88:1b:00:00:00:80
 Data : 00:00:00:00:00:00:03:e8:ea:60:13:88:2b:f2:01:f4:01:f4:c0:a8:00:11:00:17:05:b4:12


In [ ]:
def export_Register(outputFileStr):
    udpBytes = []
    udpBytes.append('        '+'|'.join(['+'+str(x) for x in range(8)]))
    for address,length in zip([0x180,0x188,0x18b,0x190,0x198,0x1b0],[8,3,5,7,8,6]):
        # print(address.to_bytes(2,'big'),(address+length).to_bytes(2,'big'))
        # print(address.to_bytes(2,'big'))
        udpBytes.append(str(hex(address))+' = '+UDP_Neunet(length=length,address=address)[8:].hex(':'))

    with open(outputFileStr,'w') as openFile:
        openFile.writelines('\n'.join(udpBytes))

In [70]:
export_Register('NEUNET Register/testBytes.dat')

b'\x01\x80'


KeyboardInterrupt: 

# Instrument time

In [89]:
secondsSince2008 = (datetime.datetime.now() - datetime.datetime(2008,1,1,0,0)).total_seconds()
print(secondsSince2008)
secondsBytes = int(secondsSince2008).to_bytes(4,'big')
print(secondsBytes.hex(':'))
subsecondsBytes = int(secondsSince2008 % 1 * 2**8).to_bytes(1,'big')
print(subsecondsBytes.hex())

492534739.475024
1d:5b:7b:d3
79


In [110]:
(secondsBytes + subsecondsBytes + bytes([0x66])).hex(':')

'1d:5b:7b:d3:79:66'

In [63]:
def instrument_time(input=None,mode='seconds'):
    '''
    Function to convert time from bytes to seconds and the reverse.
    Time is defined as seconds since 2008

    Parameters
    ---------
    input : 5 bytes representing time, optional
        If input is None, the current time will be output as 5 bytes
    mode : str
        'seconds' : output time in seconds
        'datetime' : output time in datetime

    '''
    if input is None:
        secondsSince2008 = (datetime.datetime.now() - datetime.datetime(2008,1,1,0,0)).total_seconds()
        # print(secondsSince2008)
        secondsBytes = int(secondsSince2008).to_bytes(4,'big')
        subsecondsBytes = int(secondsSince2008 % 1 * 2**8).to_bytes(1,'big')
        # print(subsecondsBytes[0])
        return secondsBytes + subsecondsBytes
    else:
        intSeconds = int.from_bytes(input[:4],'big')
        intSubSeconds = input[-1]
        secondsSince2008 = intSeconds + (intSubSeconds / 2**8)
        if mode == 'seconds':
            return secondsSince2008
        elif mode == 'datetime':
            return datetime.datetime(2008,1,1) + secondsSince2008


In [157]:
testTime = instrument_time()

492536497.797884
204


In [145]:
print(testTime.hex(':'))
int.from_bytes(testTime[0:4],'big')
testTime[-1]

1d:5b:7d:4a:99


153

In [146]:
153 / (2**8)

0.59765625

In [153]:
import SeanFunctions as sf

In [155]:
sf.math.fourierbesseltransform?

Signature: sf.math.fourierbesseltransform(q, int1, unpack=None)
Docstring:
Discrete fourier bessel transform for conversion between S(Q) and g(r)

Parameters
--------
q : array_like
    Input array of Q (or r) values
int1 : array_like
    Input array of the structure factor S values (or PDF values)
unpack : bool, optional
    If unpack is True, the result will be output as a tuple to 
    more easily define separate variables from the result

Returns 
--------
r : ndarray
    array of the real space distance (or Q) values.
ft : ndarray
    fourier transform result. Returns the PDF (or S) values.
File:      ~/mambaforge/lib/python3.11/site-packages/SeanFunctions/math.py
Type:      function

In [159]:
instrument_time(testTime)

492536497.796875